## CAFIR

CAFIR significa Cadastro de Imóveis Rurais. Através do site de [dados abertos do Governo Federal](https://dados.gov.br/home) é possível obter vários dados relacionados a tais imóveis, com exceção de informações protegidas pela Lei Geral de Proteção de Dados.

### Scraping

Para obtenção dos [dados](https://dadosabertos.rfb.gov.br/CAFIR/) será utilizado o arquivo na pasta Scripts chamado [scrap_csv.py](Scripts/scrap_csv.py), o qual irá baixar os CSVs referentes a dados Estaduais e os unirá em um único CSV.

### Limpeza de dados

O arquivo CSV resultante tem uma divisão fixa para o tamanho de cada coluna, muitos caracteres em brancos, datas com formato não padronizado e caracteres não-alfanuméricos, então é importante fazer uma "higiene" para facilitar sua inserção em bancos de dados e a visualização de seus dados.
O layout do CSV pode ser consultado também no site de [dados abertos](https://dadosabertos.rfb.gov.br/CAFIR/Layout%20Campos%20Dados%20Abertos%20Cafir.pdf), o qual utilizei como parâmetro para trocar o tamanho fixo de coluna por separadores. 
Para a limpeza dos dados foi utilizado o script [clean_csv.py](Scripts/clean_csv.py).
O resultado foi uma diminuição de quase 50% no tamanho do arquivo, que ajuda a diminuir o tempo para as queries futuras.

### SQL

Para analisar um volume tão grandes de dados é mais eficiente a utilização de um sistema de bancos de dados SQL, nesse caso em específico o Microsoft SQL. A tabela foi criada com base no layout fornecido [aqui](https://dadosabertos.rfb.gov.br/CAFIR/Layout%20Campos%20Dados%20Abertos%20Cafir.pdf). 
